In [165]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np
import statsmodels as sp


In [166]:
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'


datasets=['HCP','LOBE']



In [167]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

print(sfc_paths)
print(fc_paths)
print(sc_paths)

{'HCP': ['results/HCP/sub-HCD0021614/func/sub-HCD0021614_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii', 'results/HCP/sub-HCD0026119/func/sub-HCD0026119_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii', 'results/HCP/sub-HCD0277447/func/sub-HCD0277447_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'], 'LOBE': ['results/LOBE/sub-007/func/sub-007_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii', 'results/LOBE/sub-009/func/sub-009_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii', 'results/LOBE/sub-010/func/sub-010_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii']}
{'HCP': ['results/HCP/sub-HCD0021614/func/sub-HCD0021614_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.p

In [168]:
sfc={}
fc={}
sc={}

num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [169]:
nib.load(sc_paths['LOBE'][0]).get_fdata().shape

(300, 300)

In [170]:
# SFC group difference
x1 = sfc['HCP']
x2 = sfc['LOBE']

In [171]:
x1.shape

(3, 300)

In [172]:
tstats,pvals,dfs = sp.stats.weightstats.ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [173]:
rejected,corr_pvals = sp.stats.multitest.fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [175]:
#this sets diag to nan to ignore it


In [184]:
# FC avg per parcel, group difference
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn


In [185]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(3, 300)

In [186]:
tstats,pvals,dfs = sp.stats.weightstats.ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [187]:
rejected,corr_pvals = sp.stats.multitest.fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [188]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,